## Categorical Focal Loss
In this assignment we will implement a categorical focal loss function with "L1" and "L2" regularization for multi-class classification problems.\
Focal Loss have several applications in problems which have inbalance datasets such as Object Detection:
you can learn more about this loss function here:
https://medium.com/swlh/focal-loss-what-why-and-how-df6735f26616

In [1]:
import tensorflow as tf

## Focal Loss Formula:
$$
FL(y_{true}, y_{pred}) = - \alpha * y_{true} * (1 - y_{pred})^ γ * log(y_{pred}) \\
l1(y_{true}, y_{pred}) = ∑|y_{pred}| \\
l2(y_{true}, y_{pred}) = \sum (y_{pred})^2 \\
total-loss = FL + l1_w * l1 + l2_2 * l2
$$

In [2]:
class CategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2, l1=0.01, l2=0.01, **kwargs):
        super(CategoricalFocalLoss, self).__init__(**kwargs)
        #Code Here
        self.alpha = alpha
        self.gamma = gamma
        self.l1 = l1
        self.l2 = l2

    def call(self, y_true, y_pred):
        fl = -self.alpha * y_true * tf.math.pow((1 - y_pred), self.gamma) * tf.math.log(tf.clip_by_value(y_pred, 1e-8, 1.0))
        fl = tf.reduce_sum(fl, axis=-1) # sum over classes
        fl = tf.reduce_mean(fl) # mean over batch
        l1_term = self.l1 * tf.reduce_sum(tf.abs(y_pred))
        l2_term = self.l2 * tf.reduce_sum(tf.square(y_pred))
        total_loss = fl + l1_term + l2_term
        return total_loss

In [3]:

def build_model(dense_units, input_shape=(224, 224) + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [5]:
import tensorflow_datasets as tfds
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN, data_dir='data/')

# Initialize VGG with the number of classes
model = build_model(dense_units=256)

# Compile with losses and metrics
model.compile(optimizer='adam', loss=CategoricalFocalLoss(), metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., tf.cast(tf.one_hot(features['label'], depth=2), tf.float32)

# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling data/cats_vs_dogs/incomplete.3Z0XLV_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%|          | 0/23262 …

Dataset cats_vs_dogs downloaded and prepared to data/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the custom VGG model
model.fit(dataset, epochs=10)

Epoch 1/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 42s 49ms/step - accuracy: 0.5661 - loss: 0.5667
Epoch 2/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - accuracy: 0.7314 - loss: 0.5190
Epoch 3/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 32s 43ms/step - accuracy: 0.7985 - loss: 0.5167
Epoch 4/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - accuracy: 0.8605 - loss: 0.5143
Epoch 5/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - accuracy: 0.9179 - loss: 0.5122
Epoch 6/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - accuracy: 0.9404 - loss: 0.5112
Epoch 7/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 46ms/step - accuracy: 0.9669 - loss: 0.5101
Epoch 8/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.9806 - loss: 0.5094
Epoch 9/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - accuracy: 0.9870 - loss: 0.5090
Epoch 10/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 33s 45ms/step - accuracy: 0.9919 - loss: 0.5087
